In [5]:
import warnings
import pandas as pd
import numpy as np
from glob import iglob

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
data_path = r'C:\Users\유동환\OneDrive\Desktop\머신러닝데이터/data\*.csv' #분석할 데이터셋들의 파일 경로

In [ ]:
columns_to_extract = [ #원본 dataset에서 추출할 column들의 리스트
    'gameDuraton',
    'blueWins',
    'blueFirstBlood',
    'blueFirstTower',
    'blueFirstBaron',
    'blueFirstDragon',
    'blueFirstInhibitor',
]

In [ ]:
columns_to_compare = [ #블루팀의 기록와 레드팀의 기록의 차이를 계산해야 하는 column들
    'DragonKills',
    'BaronKills',
    'InhibitorKills',
    'TowerKills',
    'WardPlaced',
    'Wardkills',
    'Kills', #데스는 불필요 (A팀의 데스 == B팀의 킬)
    'Assist',
    'ChampionDamageDealt',
    'TotalGold',
    'TotalMinionKills',
    'AvgLevel',
    'KillingSpree',
    'ObjectDamageDealt',
    "AvgKDA", #여기서부터는 직접 유도하는 column
    'SustainedWards',
]

In [ ]:
for column in columns_to_compare[:14]: #각 팀별로 성적을 추출해야 하는 column들
    columns_to_extract.append(f'blue{column}') #[:14]인 이유는 [14:]는 추출한 다른 column들을 기반으로 획득해야 하는 데이터값이기 때문
    columns_to_extract.append(f'red{column}')

In [ ]:
df_list = [] #각 파일별로 생성한 dataframe을 넣어주며 나중에 하나의 거대한 dataframe으로 취합

In [ ]:
game_length_bins = list(range(20, 51, 10)) #게임 시간 20분 이상부터 50분까지의 경기만을 분석
game_length_bins_label = [str(x)+"분이상 "+str(x+10)+"분미만" for x in game_length_bins] #game duration column을 일정 단위로 나누는 방식으로 변경한 후 각 column이 가질 값

In [4]:
def handle_df(df):
    df = df.rename(columns ={ #dataset의 column name 오타 수정
        'gameDuraton' : 'gameDuration',
    })
    df['gameDuration'] = df['gameDuration'] / 60 #게임 시간 column 분 단위로 변경
    #게임에서 영향을 주는 요소는 크게 두 가지가 있다.
    # 첫 번째로 롤은 상대적인 우위에 의해 승자가 결정된다. 따라서 절대적인 수치가 아니라 각 팀간의 격차를 파악한다.
    # 두 번째로 동일한 데이터 수치가 게임 경과 시간에 따라 다른 의미를 가지기도 한다.
    # 따라서 게임 시간 10분 단위로 게임들을 그룹지어준 뒤
    # 그룹별로 각 게임에서 양팀이 보여준 통계 차이를 기반으로 승리에 영향을 끼치는 요인들을 분석한다.
    
    df['redAvgKDA'] = (df['redKills'] + df['redAssist']) / df['blueKills'] #평균 KDA (Kill Death Assist)
    df['blueAvgKDA'] = (df['blueKills'] + df['blueAssist']) / df['redKills']
    df['redSustainedWards'] = df['redWardPlaced'] - df['blueWardkills'] #상대팀에게 파괴되지 않고 유지된 와드 수
    df['blueSustainedWards'] = df['blueWardPlaced'] - df['redWardkills']

    for i, column in enumerate(columns_to_compare): 
        newColumnName = column[0].lower() + column[1:] + 'Diff' # 각 팀간 성적을 비교할 수 있는 지표에 대해 격차를 계산하는 column 새롭게 생성
        
        if i < 3:#용, 바론, 억제기는 지표값이 크지 않기 때문에 단순히 차이로 계산
            df[newColumnName] = df[f'blue{column}'] - df[f'red{column}']
        else:
            #그 외는 지표의 수치와 둘 간의 격차를 모두 고려하기 위해
            #(블루팀의 성적 - 레드팀의 성적) / (블루팀의 성적 + 레드팀의 성적)으로 계산한다
            #분모를 합으로 처리한 것은 분모가 0이 되는 것을 방지
            df[newColumnName] = np.where(
                ((df[f'blue{column}'] == 0) & (df[f'red{column}'] == 0)),
                0, #양 팀 모두 해당 지표값이 0이면 그냥 0으로 계산 (차이가 없음) -> 분모가 0이 되는 것 방지
                (df[f'blue{column}'] - df[f'red{column}']) / (df[f'blue{column}'] + df[f'red{column}']) #두 팀 중 하나라도 지표의 값이 0이 아닌 경우
            )
        df = df.drop(columns=[ #격차를 획득한 후에 각 팀의 성적은 필요없으므로 버림
            f'blue{column}', f'red{column}'
        ])

    df['gameDuration'] = pd.cut(df["gameDuration"], game_length_bins, right=False, labels=game_length_bins_label[:-1]) #gameDuration column을 game시간 5분 단위로 분류
    df = df.dropna(axis=0) #게임 시간이 20분 미만이거나 50분 이상인 게임들은 특수한 경우라 판단하여 분석 범위에서 제외
    df_list.append(df)
    
for file in iglob(data_path, recursive=True):
    df = pd.read_csv(file, header = 0, engine="python")[columns_to_extract]
    handle_df(df) #각 파일별로 dataframe 전처리 후 df_list에 넣어줌

In [ ]:
final_df = pd.concat(df_list) #모든 파일의 dataframe이 결합된 최종 dataframe

In [ ]:
x_data = final_df.drop(columns=['blueWins']) #종속변수 (블루팀의 게임 승리 여부)
y_data = final_df['blueWins'] #독립변수